In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
ruta_base = "anid/datos-separados/"
ruta_final = "anid/finales/"

## Unificar datos

In [3]:
xls1 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_1982_2002.xlsx")
xls2 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_2003_2022.xlsx")
xls3 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_2023_2032.xlsx")

xls_unificado = pd.concat( [xls1, xls2, xls3] )
xls_unificado.to_excel(f"{ruta_final}proyectos-unificados-anid.xlsx")

## Filtro para buscar temas antarticos

In [4]:
tmp_array = xls_unificado.to_numpy()
final_list = []

for row in tmp_array:
    if type(row[22]) != float and type(row[8]) != float:
        #Solo buscar temas antarticos
        if ("ANTARC" in row[22].upper() or "ANTART" in row[22].upper() or "ANTARC" in row[8].upper() or "ANTART" in row[8].upper()):
            final_list.append( row )
        else:
            pass

In [5]:
dt = pd.DataFrame( np.array(final_list), columns=['N', 'CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION',
       'DURACION_MESES', 'TIPO_BENEFICIARIO', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'NOTA_MONTO', 'MONEDA', 'PALABRAS_CLAVES'])
dt.to_excel(f"{ruta_final}proyectos-primer-filtro.xlsx")

## Agregar columna de tema antartico al excel completo

In [4]:
xls1 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_1982_2002.xlsx")
xls2 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_2003_2022.xlsx")
xls3 = pd.read_excel(f"{ruta_base}BDH_PROYECTOS_2023_2032.xlsx")

xls = pd.concat( [xls1, xls2, xls3] )

In [5]:
xls_antartico = pd.read_excel(f"anid/filtrado-amano.xlsx")['CODIGO_PROYECTO'].tolist()

In [6]:
todos_codigos_proyecto = xls['CODIGO_PROYECTO'].tolist()
lista_booleana = []

for codigo in todos_codigos_proyecto:
    if codigo in xls_antartico:
        lista_booleana.append('SI')
    else:
        lista_booleana.append('NO')

In [7]:
tmp_array = xls.to_numpy().T.tolist()
tmp_array.append( lista_booleana )
tmp_array = np.array( tmp_array ).T

In [21]:
dt = pd.DataFrame( tmp_array, columns=['N', 'CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION',
       'DURACION_MESES', 'TIPO_BENEFICIARIO', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'NOTA_MONTO', 'MONEDA', 'PALABRAS_CLAVES', 'ANTARTIC'])
dt = dt.drop(columns=['N', 'AGNO_CONCURSO', 'DURACION_MESES', 'NOTA_MONTO', 
       'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION', 'TIPO_BENEFICIARIO'])

In [33]:
tmp_list=[]

for row in dt.to_numpy().tolist():
    codigo, subdir, programa, instrumento, nombre_concurso, agno, nombre_proyecto, area_ocde, responsable, sexo, insti, macro, region, monto, moneda, palabras, antartic = row
    
    if pd.isna(monto) or monto=='NO SOLICITA A CONICYT':
        monto=0.0
    else:
        if type(monto) == str:
            monto = float( monto.replace(',', '.') )

    if moneda == 'EUROS':
        #monto = monto.replace(',', '.')
        monto = float(monto)*1042.36/1000.0

    tmp_list.append([codigo, subdir, programa, instrumento, nombre_concurso, agno, nombre_proyecto, area_ocde,
            responsable, sexo, insti, macro, region, monto, palabras, antartic])        
    
    new_dt = pd.DataFrame( np.array(tmp_list), columns=['CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'PALABRAS_CLAVES', 'ANTARTIC'])

KeyboardInterrupt: 

In [30]:
dt['MONTO_ADJUDICADO'] = dt['MONTO_ADJUDICADO'].fillna(0.0)

In [31]:
dt['CODIGO_PROYECTO'] = dt['CODIGO_PROYECTO'].astype(str)
dt['MONTO_ADJUDICADO'] = dt['MONTO_ADJUDICADO'].astype(float)
dt['AGNO_FALLO'] = dt['AGNO_FALLO'].astype(float)

ValueError: could not convert string to float: '4689,75'

In [16]:
dt.to_excel(f"{ruta_final}datos-totales-columna-nueva.xlsx")